# Solvation
VeloxChem has two ways to compute the Gibbs energy of solvation - implicit CPCM and explicit solvation using alchemical free energy perturbation.

In [ ]:
import veloxchem as vlx

In this example we will solvate a phenolate anion using both methods. First we define the molecule object and set the charge to -1.

In [ ]:
molecule = vlx.Molecule.read_smiles('c1ccccc1[O-]')
molecule.set_charge(-1)
molecule.show()

### Implicit solvation
To compute the solvation free energy using CPCM we simply compute the gas phase energy first, then use ```.solvation_model = 'cpcm'``` on the scf driver, and then compute the scf again. The free energy of solvation will be the difference between the cpcm_results and scf_results.

In [ ]:
basis = vlx.MolecularBasis.read(molecule, 'def2-svp')
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.xcfun = "b3lyp"
scf_results = scf_drv.compute(molecule, basis)
scf_drv.solvation_model = 'cpcm'
cpcm_results = scf_drv.compute(molecule, basis)

In [ ]:
Gsolv = cpcm_results['scf_energy'] - scf_results['scf_energy']

print('Experimental solvation free energy:')
print('-72.5 kcal/ mol')
print(f"{-72.5*4.184:.1f} kJ/ mol")
print(" ")
print('CPCM solvation free energy:')
print(f"{Gsolv*627.5:.1f} kcal/mol")
print(f"{Gsolv*2625.5:.1f} kJ/mol")

### Explicit Solvation
To compute the solvation using alchemical free energy perturbation protocol you need the two lines below. This will generate a force field with RESP charges (HF/6-31G*), solvate it in a box with 2 nm padding and run every lamda for 1 ns.

To run the cells below remove the """ in the cells. These computations can be a bit slow so do not run them if you do not have a good GPU.

In [ ]:
"""
explicit_solvation = vlx.SolvationFepDriver()
explicit_solvation_result = explicit_solvation.compute(molecule, solvent='cspce')
"""

If you want to try it a bit faster you can lower the padding and simulation time per lambda. If you have a GPU it will be significantly faster if you specify a different platform than CPU (OpenCL, HIP, CUDA).

In [ ]:
"""
explicit_solvation = vlx.SolvationFepDriver()
explicit_solvation.padding = 1.1
explicit_solvation.platform = "OpenCL"
explicit_solvation.num_steps = 100000
explicit_solvation_result = explicit_solvation.compute(molecule, solvent='cspce')
"""

In [ ]:
"""solvation = explicit_solvation_result[-1]-molecule.get_charge()*57.7/4.184 #correction term added for the potential in taking the ion across the interface https://doi.org/10.1021/acs.jpclett.7b01125
print(f"{solvation:.1f} kcal/mol")
print(f"{solvation*4.184:.1f} kJ/mol")
"""

You can also specify your own force field for both the solute and solvent. Here I have used a forcefield with RESP charges computed with DFT and CPCM.

In [ ]:
"""
ff_gen = vlx.MMForceFieldGenerator()
ff_gen.create_topology(molecule, basis, cpcm_results)
explicit_solvation2 = vlx.SolvationFepDriver()
explicit_solvation2.padding = 1.1
explicit_solvation2.platform = "OpenCL"
explicit_solvation2.num_steps = 100000
explicit_solvation_result2 = explicit_solvation2.compute(molecule, ff_gen_solute=ff_gen, solvent='spce')
"""

In [ ]:
"""
solvation2 = explicit_solvation_result2[-1]-molecule.get_charge()*57.7/4.184
print(solvation2)
print(solvation2*4.184)
"""